In [77]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
from pandas_datareader import data as pdr
from bs4 import BeautifulSoup as bs
from scipy import stats
import datetime

In [78]:
tick = pd.read_csv("all_stocks")

In [83]:
final = pd.DataFrame(columns=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy'])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = tick["0"][i]
        stock_info = yf.Ticker(ticker)

        # Get the income statement for the stock
        income_statement = stock_info.income_stmt

        # Get the amortization expense from the income statement

        # Print the amortization expense
        print(income_statement.index)
        #In info: 
            #We have price to book
            #We have price to sale 12 months
            #We have enterprise value
            #We have gross profit
        #In income_stmt: 
        #EBIDTA

        #Ratio endettement (liabilities/assets)
        #Working capital (currentAssets/currentLiability) Voir si depense sont capapble d'etre couverte dans la prochaine annee

    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    break

    '''    
    try: 
        pass
    except IndexError: 
        continue
    #Change to keep only last 2 days!!! get date
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy']), 
        ignore_index = True
    ) 
    '''

Producing data for:  AOS
Index(['Total Revenue', 'Operating Revenue', 'Cost Of Revenue', 'Gross Profit',
       'Operating Expense', 'Selling General And Administration',
       'Operating Income', 'Net Non Operating Interest Income Expense',
       'Interest Expense Non Operating', 'Other Income Expense',
       'Special Income Charges', 'Restructuring And Mergern Acquisition',
       'Other Non Operating Income Expenses', 'Pretax Income', 'Tax Provision',
       'Net Income Common Stockholders', 'Net Income',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations',
       'Diluted NI Availto Com Stockholders', 'Basic EPS', 'Diluted EPS',
       'Basic Average Shares', 'Diluted Average Shares', 'Total Expenses',
       'Net Income From Continuing And Discontinued Operation',
       'Normalized Income', 'Interest Expense', 'Net Interest Income', 'EBIT',
       'Reconciled Cost Of Revenue', 'Reconciled Depreciation',
       'Net Income From Cont

In [5]:
final.to_csv("value_invest_advanced_sheet")